# import libraries

In [ ]:
import torchvision
import torch
from PIL import Image

size = (128, 128)
transform = torchvision.transforms.Compose([torchvision.transforms.Resize(size), torchvision.transforms.ToTensor()])
train_dataset = list(torchvision.datasets.Flowers102("./flowers", "train", transform=transform, download= True))
test_dataset = list(torchvision.datasets.Flowers102("./flowers", "test", transform=transform, download= True))

def visualize_image(img: torch.Tensor) -> Image.Image:
    return Image.fromarray((img.permute(1,2,0)* 255).to(torch.uint8).numpy())


# Visualize

In [ ]:
import matplotlib.pyplot as plt
f, ax = plt.subplots(4, 10, figsize = (10, 5))
for i, (im,l) in enumerate(list(train_dataset)[:40]):
    ax[i // 10, i%10].imshow(visualize_image(im))
    ax[i // 10, i%10].set_title(l)
    ax[i//10, i%10].axis('off')

# change the data to tensors

In [15]:
train_images = torch.stack([im for im, _ in train_dataset], dim = 0)
train_labels = torch.tensor([l for _, l in train_dataset])

# model + loss function

In [27]:
model = torch.nn.Linear(3*128*128, 1)
loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.0001)

# training

In [ ]:
train_images = train_images[train_labels <= 1]
train_labels = train_labels[train_labels <= 1]

for epoch in range(100):
    #model pred
    pred_label = model(train_images.view(-1, 128*128*3))
    #loss calc
    loss_train = loss(pred_label.view(-1), train_labels.float())
    #reset gradient
    optimizer.zero_grad()

    loss_train.backward()
    optimizer.step()
    print(f'epoch: {epoch}, loss: {loss_train}')   

# accuracy

In [35]:
test_images = torch.stack([im for im, _ in test_dataset], dim = 0)
test_labels = torch.tensor([l for _, l in test_dataset])

test_images = test_images[test_labels<=1]
test_labels = test_labels[test_labels<=1]
pred_label = model(test_images.view(-1, 128 * 128 * 3))
print(loss(pred_label.view(-1), test_labels.float()).item())

0.30193892121315


# binary cross entropy

In [39]:
train_images = torch.stack([im for im, _ in train_dataset], dim = 0)
train_labels = torch.tensor([l for _, l in train_dataset])

train_images = train_images[train_labels <=1]
train_labels = train_labels[train_labels <=1]

loss = torch.nn.BCELoss()

for epoch in range(10):
    pred = model(train_images.view(-1, 128 * 128 * 3))

    loss_train = loss(pred, train_labels)
    